In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
import re
import math
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd


In [2]:
#Funcion para obtener tabla para un pestaña de una solicitud de 
#información.
def obtener_tabla():
    page_source = driver.page_source
    table = pd.read_html(page_source)
    lista = table[0]
    columns = ['Nombre Comercial','Cámara', 'Entidad Federativa', 'Actividad']
    lista.columns = columns  
    lista.loc[:,'Entidad Federativa'] = 'Ciudad de México'
    lista.loc[:,'Actividad'] = lista_actividades[posicion_actividad]
    lista = lista[['Nombre Comercial','Cámara', 'Actividad','Entidad Federativa']]
    return lista

In [3]:
driver = webdriver.Chrome('/Users/lupita/Documents/IronHack/complementos/chromedriver')

In [ ]:
#Obteniendo la lista de Cámaras
driver.find_element_by_xpath('//*[@id="vs1__combobox"]').click()
time.sleep(5)
lista_1 = driver.find_elements_by_id('vs1__listbox')
camaras = [element.text.split('\n') for element in lista_1]
ActionChains(driver).send_keys(Keys.RETURN).perform()
print(camaras )


In [ ]:
lista_camaras = pd.DataFrame(camaras[0], columns=['Camaras'])
lista_camaras.to_csv(r'ListaCamaras.csv', encoding="utf-8")
lista_camaras.head()

In [6]:
#Obteniendo la lista de Actividades
driver.find_element_by_xpath('//*[@id="vs2__combobox"]').click()
lista_2 = driver.find_elements_by_id('vs2__listbox')
actividades = [element.text.split('\n') for element in lista_2]
ActionChains(driver).send_keys(Keys.RETURN).perform()
lista_actividades = actividades[1]
print(len(lista_actividades))
type(lista_actividades)

1098


list

In [ ]:
#Convertir la lista de actividades a un csv
sectores=pd.DataFrame(lista_actividades, columns=['Actividad'])
sectores.to_csv(r'Actividades.csv', encoding="utf-8")

In [7]:
#Obteniendo la lista de entidades
driver.find_element_by_xpath('//*[@id="vs3__combobox"]').click()
lista_3 = driver.find_elements_by_id('vs3__listbox')
entidades = [element.text.split('\n') for element in lista_3]
ActionChains(driver).send_keys(Keys.RETURN).perform()

In [8]:
lista_entidades=pd.DataFrame(entidades[1][1:], columns=['Entidades'])
lista_entidades.to_csv('ListaEntidades.csv',encoding="utf-8")

In [9]:
#Seleccionando la Ciudad de México como entidad
driver.find_element_by_xpath('//*[@id="vs3__combobox"]').click()
posicion_entidad = entidades[1].index('Ciudad de México')
for i in range(posicion_entidad):
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
ActionChains(driver).send_keys(Keys.RETURN).perform()

In [10]:
#Obteniendo el indice para una determinada actividad
posicion_actividad = lista_actividades.index('Hoteles con otros servicios integrados')
posicion_actividad

741

In [11]:
#Seleccionando una actividad ingresando el nombre de la actividad
elemen = driver.find_element_by_xpath('//*[@id="vs2__combobox"]')
elemen.click()
ActionChains(driver).send_keys(lista_actividades[posicion_actividad]).perform()
ActionChains(driver).send_keys(Keys.RETURN).perform()
driver.find_element_by_id('botonGuardar').click()

In [12]:
#Verificando cuantos registros encontró para esa actividad. Si no
#encuentra debe seguir con el siguiente sector. 
registros = driver.find_elements_by_css_selector('h3.text-secondary')
leyenda_registros = [element.text for element in registros]
numero =int(re.search('\d+',leyenda_registros[0]).group())
if numero>0:
    print('a buscar')
else:
    print("Continua con la siguiente actividad")
#Calcula el número de paginas que tiene ese sector. 
paginas =math.ceil(numero/10)
print(numero, paginas)

a buscar
649 65


In [13]:
#Recopilando la información de la busqueda por actividad.
#Obteniendo todas la tablas de las pestañas y uniendola en una sola tabla
lista_empresas=obtener_tabla()
for i in range(paginas-1):
    driver.find_element_by_xpath('//*[@id="__BVID__70"]/div/ul/li[8]/button').click()
    empresas_nuevas=obtener_tabla()
    lista_empresas= pd.concat([lista_empresas,empresas_nuevas])
lista_empresas = lista_empresas.reset_index(drop=True)
lista_empresas
lista_empresas.to_csv('lista_empresas0.csv',encoding="utf-8")
#lista1['Nombre Comercial']


In [14]:
lista_empresas

,Nombre Comercial,Cámara,Actividad,Entidad Federativa
0,VP LIVE,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
1,MARKA ENTERTAINMENT GROUP,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
2,NH AEROPUERTO,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
3,CHASTEL,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
4,HOTEL BLANCO,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
...,...,...,...,...
644,"RIO LERMA 162, S.A. DE C.V.",CANACOPE SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
645,"OPERADORA HOTELERA BRISAS, S.A. DE C.V.",CANACOPE SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
646,HOTEL PEGASO,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
647,HOTEL CANADA,CANACO SERVYTUR CD DE MEXICO,Hoteles con otros servicios integrados,Ciudad de México
